In [2]:
# !git clone https://github.com/Shorstko/mai_python_2022
# %cd mai_python_2022/Homework

In [132]:
!mkdir res

In [139]:
# В ЭТОМ ДЗ ВЫ БУДЕТЕ АНАЛИЗИРОВАТЬ ДАННЫЕ ОБ АВИАПРОИСШЕСТВИЯХ С УЧАСТИЕМ МОДЕЛЕЙ ДРОНОВ ИЗ ВАШИХ ИСХОДНЫХ ДАННЫХ В .JSON

# =====================================
# ЗАДАНИЕ 1: Классы - декораторы
# =====================================
# TODO 1-1 - Добавить атрибут incidents типа список
# Возьмите код из предыдущего ДЗ
# Добавьте в класс MultirotorUAV атрибут incidents и внесите туда информацию обо всех найденных происшествиях для этой модели
# Не забудьте, что атрибут добавляется при помощи декоратора


class Aircraft:
    def __init__(self, weight):
        self._weight = weight

class UAV:
    def __init__(self):
        self._has_autopilot = True
        self.__missions: list([list()])

	# напишите код для декоратора атрибута _missions
	
    @property
    def get_miss(self):
        return self.__missions

    @get_miss.setter
    def add_miss(self, mission):
        self.__missions = mission
 
    # напишите публичный метод count_missions
    def count_missions(self):
        return len(self.missions)

    def get_missions(self):
        return self.missions

class MultirotorUAV(Aircraft, UAV):
    def __init__(self, weight, model, brand):
        super().__init__(weight)
        UAV.__init__(self)
        self.__weight = weight
        self.__model = model
        self.__brand = brand
        self.__incidents:list([str]) = []

	# напишите публичный метод get_info
    def get_info(self):
        print(f'масса: {self.__weight}, производитель: {self.__brand}, количество миссий: {self.count_missions()}')#

	# напишите публичный метод get_model
    def get_model(self):
        return self.__model

    # напишите код декоратора для атрибута incidents. Не забудьте сначала добавить приватный атрибут в класс
    @property
    def incidents(self):
        return self.__incidents
    # напишите публичный метод add_incident, который добавляет инцидент в список инцидентов для данной модели дрона
    @incidents.setter
    def add_incident(self, incident):
        self.__incidents.append(incident)

	# напишите публичный метод save_data, который сохраняет информацию о дроне в файл json
    def save_data(self):
        d = {}
        d['model'] = self.__model
        d['weight'] = self.__weight
        d['brand'] = self.__brand
        d['missions'] = self.get_missions()
        d['incidents'] = self.__incidents
        with open(f"/content/mai_python_2022/Homework/res/{self.__class__.__name__}_{self.__model}.json", "w") as f:
            json.dump(d, f, ensure_ascii=False, indent=2)
# ВАШ КОД из предыдущего ДЗ, необходимый для решения этого ДЗ (чтение данных о пилотах, сбор информации о дронах и пр.):

import json
from pprint import pprint

with open("/content/mai_python_2022/Homework/pilot_path.json") as f:
	json_data = json.load(f)
 

drone_catalog = {
	"DJI Mavic 2 Pro": {"weight":903, "brand":"DJI"},
	"DJI Mavic 2 Zoom": {"weight":900, "brand":"DJI"},
	"DJI Mavic 2 Enterprise Advanced": {"weight":920, "brand":"DJI"},
	"DJI Inspire 2": {"weight":1500, "brand":"DJI"},
	"DJI Mavic 3": {"weight":1000, "brand":"DJI"}
}

result = list()

for i, dron in enumerate(drone_catalog):
	result.append(MultirotorUAV(drone_catalog[dron]['weight'], dron, drone_catalog[dron]['brand']))
	tmp = list()
	for pilot in json_data:
		for j in json_data[pilot]['missions']:
			if j['drone'] == dron:
				tmp.append(j['mission'])
	result[i].missions=tmp

In [140]:
# =====================================
# ЗАДАНИЕ 2: Файлы - CSV
# =====================================
# TODO 2-1 - Загрузите информацию об авиапроисшествиях из файла csv
# Проверьте по моделям (названия моделей возьмите из экземпляров класса MultirotorUAV), какие из них участвовали в авиапроисшествиях

# ВАШ КОД чтения данных из файла:

read_csv = list()

with open("/content/mai_python_2022/Homework/faa_incidents.csv") as f:
    lines = f.readlines()
    print(lines[0])
    lines[0] = lines[0].split('\n')[0].split(',')
    for i in range(1, len(lines)):
        line = lines[i]
        read_csv.append(line.split('"'))

Data Source,Report Number,Local Date,City,State,Airport Name,Event Type,Mid Air Collision,Aircraft Damage,Registration Nbr,Total Aboard,Fatalities,Injuries,Details



In [141]:
# =====================================
# ЗАДАНИЕ 3: Классы
# =====================================
# TODO 3-1 - Для каждой модели дрона добавьте в экземпляр класса информацию об авиапроисшествиях, в которых участвовала эта модель
# Информацию сохраните в атрибут incidents (используйте декораторы)
# Подсказка: вот так вы получаете названия модели для каждого экземпляра класса MultirotorUAV
# Экземпляры все так же находятся в списке (например, drones_cls_list)

for j, drone_cls in enumerate(result):
    drone = drone_cls.get_model()
    drone = drone[4:].upper()
    for iter in read_csv:
        if len(iter) > 1:
            if drone in iter[1].upper():
                result[j].incidents.append(iter[1])
                print(drone)
        else:
            if drone in iter[0].split(',')[13].upper():
                result[j].incidents.append(iter[0].split(',')[13])
                print(drone)

MAVIC 2 ZOOM
INSPIRE 2
INSPIRE 2
INSPIRE 2


In [142]:
# TODO 3-2 - Добавьте в класс MultirotorUAV публичный метод save_data, который сохраняет статистику по дрону в файл
# Внимание! Метод save_data не принимает параметры. Название файла сформируйте как: название класса + название модели + расширение .json
# например: "MultirotorUAV_DJI Mavic 2 Pro.json"
# Подсказка: название класса вы можете получить вот так: self.__class__.__name__
# используйте ключи: "model", "weight", "brand", "missions", "incidents"
# например: {"model":"DJI Inspire 2", "weight": 1500, "info": "...", "manufacturer": "DJI", "missions": [], "incidents": []}

# ВАШ КОД - допишите код в объявлении класса

In [143]:
# =====================================
# ЗАДАНИЕ 4: Регулярные выражения
# =====================================
# TODO 4-1 - Выведите на экран собранную информацию по инцидентам по каждому дрону в таком виде:
# модель: инцидентов - количество
# 1 - краткий текст инцидента*
# полный текст инцидента
# * - краткий текст инцидента получайте следующим образом: в исходном тексте инцидента найдите модель, например, INSPIRE 2,
# и выведите все предложения, в которых встречается упоминание этой модели
# Подсказка 1: Полностью готовый код есть в лекции про регулярные выражения (пример про перелетных птиц).
# Ваши изменения: а) вставить вместо "зим" название модели дрона, б) поменять язык поиска на английский
# Подсказка 2: не забывайте использовать флаг re.I для игнорирования регистра символов
# Подсказка 3: перед тем, как искать, уберите из названия модели название производителя
# Подсказка 4: лучше не используйте re.compile. Для этого случая работает не очень

# РЕЗУЛЬТАТ:
# mavic 2 enterprise advanced: инцидентов - 0
# mavic 2 pro: инцидентов - 0
# mavic 3: инцидентов - 0
# mavic 2 zoom: инцидентов - 1
# 1 - ON JULY 15, 2020 AT 1050 EDT, A DJI, MAVIC 2 ZOOM L1Z UAS, SERIAL # 0M6TG85R0A04ZP, UA FA REGISTRATION # FA3RE7RNWP, REGISTERED TO ^PRIVACY DATA OMITTED^ (PIC), REMOTE PILOT CERTIFICATE ^PRIVACY DATA OMITTED^, LOST CONTROLLED FLIGHT IN THE AREA OF ^PRIVACY DATA OMITTED^ AND HIT A BLACK NISSAN PICKUP TRUCK BEARING ^PRIVACY DATA OMITTED^ TRAVELING ALONG TAMIAMI TRAIL IN NORTH PORT CAUSING PROPERTY DAMAGE.
# ON JULY 15, 2020 AT 1050 EDT, A DJI, MAVIC 2 ZOOM L1Z UAS, SERIAL # 0M6TG85R0A04ZP, UA FA REGISTRATION # FA3RE7RNWP, REGISTERED TO ^PRIVACY DATA OMITTED^ (PIC), REMOTE PILOT CERTIFICATE ^PRIVACY DATA OMITTED^, LOST CONTROLLED FLIGHT IN THE AREA OF ^PRIVACY DATA OMITTED^ AND HIT A BLACK NISSAN PICKUP TRUCK BEARING ^PRIVACY DATA OMITTED^ TRAVELING ALONG TAMIAMI TRAIL IN NORTH PORT CAUSING PROPERTY DAMAGE. THE UAS WAS FLOWN ON A RECREATIONAL FLIGHT OVER A CONSTRUCTION SITE AT ^PRIVACY DATA OMITTED^, USING AN AUTOMATIC FREQUENCY SELECTION FEATURE THAT RANGED FROM 2.400 - 2.4835 GHZ; 5.725 - 5.850 GHZ. WEATHER CONDITIONS WERE CLEAR AND ARE NOT CONSIDERED A FACTOR. THE UAS WAS DESTROYED AND THE PROPERTY DAMAGE WAS GREATER THAN $500. THERE WERE NO PERSONAL INJURIES. THE FLIGHT ORIGINATED FROM ^PRIVACY DATA OMITTED^, EARLIER THAT DAY. WHILE THIS INCIDENT MEETS FAA UAS ACCIDENT CRITERIA, IT DOES NOT MEET THE NTSB?S UAS ACCIDENT CRITERIA. THE NTSB WOULD NOT ISSUE A NTSB ACCIDENT NUMBER FOR THIS EVENT. THEREFORE, THIS EVENT WILL BE CLASSIFIED AN INCIDENT.
# inspire 2: инцидентов - 3
# 1 - RPIC WAS OPERATING A DJI INSPIRE 2 WITH A CAMERA/GIMBLE COMBINATION SET UP. DJI INSPIRE 2 (SN 09YDDCQL040384) CURRENT FAA SUAS REGISTRATION NUMBER FA343FTPWM.
# ON MAY 25, 2020, MOJAVE AIR AND SPACE PORT (KMHV) AIR TRAFFIC CONTROL TOWER (ATCT) PROVIDED VERBAL AUTHORIZATION TO ^PRIVACY DATA OMITTED^ TO PERFORM SUAS OPERATIONS IN KMHV CLASS D AIRSPACE. WIND SPEED WAS 7 KNOTS WITH GUSTS AT 14 KNOTS. WIND DIRECTION 060. ^PRIVACY DATA OMITTED^. AND KMHV HAVE A LETTER OF AGREEMENT (LOA) EFFECTIVE JANUARY 28, 2020 FOR SUAS OPERATIONS IN KMHV CLASS D AIRSPACE. ^PRIVACY DATA OMITTED^ HOLDS A CURRENT CERTIFICATE OF WAIVER OR AUTHORIZATION EFFECTIVE FROM JANUARY 31, 2019 TO SEPTEMBER 30, 2020 TO PERFORM SMALL UNMANNED AIRCRAFT SYTEM (SUAS) OPERATIONS IN CLASS D AIRSPACE AT KMHV. ^PRIVACY DATA OMITTED^ HOLDS A CURRENT REMOTE PILOT CERTIFICATE WITH A SUAS RATING ^PRIVACY DATA OMITTED^. REMOTE PILOT IN COMMAND (RPIC), ^PRIVACY DATA OMITTED^ WAS OPERATING UNDER PART 107 AT KMHV COVERING SURVEILLANCE OF THE VIRGIN ORBIT LAUNCHER ONE MISSION. RPIC WAS OPERATING A DJI INSPIRE 2 WITH A CAMERA/GIMBLE COMBINATION SET UP. DJI INSPIRE 2 (SN 09YDDCQL040384) CURRENT FAA SUAS REGISTRATION NUMBER FA343FTPWM. REGISTERED TO ^PRIVACY DATA OMITTED^. RPIC WAS OPERATING WITH ONE INEXPERIENCED VISUAL OBSERVER (VO). RPIC WAS NOT ACCUSTOMED TO OPERATING THE SMALL UNMANNED AIRCRAFT CONTROLS, MANIPULATING THE CAMERA/GIMBLE COMBINATION SET UP, AND COMMUNICATING TO ATCT VIA RADIO SIMULTANEOUSLY. RPIC WAS ALSO WORKING WITH AN INEXPERIENCED VO. RPIC BECAME TASK SATURATED AND LOST SIGHT OF THE SMALL UNMANNED AIRCRAFT. AT 1208 LOCAL TIME, THE SMALL UNMANNED AIRCRAFT STRUCK THE WEST SIDE OF KMHV ATCT. ^PRIVACY DATA OMITTED^ SUSTAINED A SUPERFICIAL HORIZONTAL LACERATION TO HIS RIGHT LOWER, INSIDE FOREARM. HE WAS TREATED ON-SCENE WITH BANDAGE AND GAUZE WRAP.
# 2 - USED RENTED DJI INSPIRE 2 DRONE.
# UAS PILOT ^PRIVACY DATA OMITTED^ REMOTE CERTIFICATE #^PRIVACY DATA OMITTED^ WAS HIRED BY PRODUCER ^PRIVACY DATA OMITTED^ TO DO SOME AERIAL SHOTS OF EL MORRO FOR A DOCUMENTARY ABOUT THE 500 YEARS OF THE CITY OF SAN JUAN ON SEPTEMBER 3RD. USED RENTED DJI INSPIRE 2 DRONE. LOST CONTROL LINK WITH DRONE ON WAY BACK CRASHED INTO EL MORRO FORT IN OLD SAN JUAN.
# 3 - AIRCRAFT IS A DJI T650A INSPIRE 2 SUAS, SERIAL # 0A0LG2J107005, REGISTRATION # FA3FTYCLFE.
# AIRCRAFT IS A DJI T650A INSPIRE 2 SUAS, SERIAL # 0A0LG2J107005, REGISTRATION # FA3FTYCLFE. FREQUENCY USED IS UNKNOWN. THE AIRCRAFT HAS TWO FREQUENCIES AVAILABLE, 2.4 AND 5.8 GHZ, BUT THE PIC DOESN'T REMEMBER WHICH ONE WAS IN USE DURING THE FLIGHT. PIC IS ^PRIVACY DATA OMITTED^, CERTIFICATE ^PRIVACY DATA OMITTED^. ^PRIVACY DATA OMITTED^ SAID THAT THE UAS EXPERIENCED AN ERROR ON ITS FIRST CALIBRATION ATTEMPT PRIOR TO LAUNCH BUT CALIBRATED CORRECTLY ON THE SECOND ATTEMPT AND THE FLIGHT CONTINUED AFTER RECORDING THE HOME POINT AT THE LAUNCH POSITION. HE SAID THE UAS WAS IN POSITIONING MODE (P-MODE) FOR THE ENTIRE FLIGHT. THE LANDING SEQUENCE WAS INITIATED MANUALLY (I.E. THE AUTOLAND FEATURE WAS NOT USED). ^PRIVACY DATA OMITTED^ SAID THAT THE UAS "TOOK OFF" WHEN IT GOT DOWN TO ABOUT 5' AGL AND FLEW INSIDE THE CPD HANGAR WHERE IT STRUCK A CPD HELICOPTER. HE SAID THAT THE UAS ACTED AS IF IT WAS "PRE-PROGRAMMED" TO FLY INTO THE HANGAR ONCE IT GOT AWAY FROM HIM.

# ВАШ КОД:

for j, drone_cls in enumerate(result):
    drone = drone_cls.get_model()
    drone = drone[4:].lower()
    if len(drone_cls.incidents) > 0:
        print(f'{drone}: инцидентов - {len(drone_cls.incidents)}')
        for i, drones in enumerate(drone_cls.incidents):
            tmp = drones.split('.')
            s = ''
            for iter in tmp:
                if drone.upper() in iter:
                    s += iter
            print(f'{i + 1} - {s}')
            print(drones.split('\n')[0])
    else:
        print(f'{drone}: инцидентов - {len(drone_cls.incidents)}')

mavic 2 pro: инцидентов - 0
mavic 2 zoom: инцидентов - 1
1 - ON JULY 15, 2020 AT 1050 EDT, A DJI, MAVIC 2 ZOOM L1Z UAS, SERIAL # 0M6TG85R0A04ZP, UA FA REGISTRATION # FA3RE7RNWP, REGISTERED TO ^PRIVACY DATA OMITTED^ (PIC), REMOTE PILOT CERTIFICATE ^PRIVACY DATA OMITTED^, LOST CONTROLLED FLIGHT IN THE AREA OF ^PRIVACY DATA OMITTED^ AND HIT A BLACK NISSAN PICKUP TRUCK BEARING ^PRIVACY DATA OMITTED^ TRAVELING ALONG TAMIAMI TRAIL IN NORTH PORT CAUSING PROPERTY DAMAGE
ON JULY 15, 2020 AT 1050 EDT, A DJI, MAVIC 2 ZOOM L1Z UAS, SERIAL # 0M6TG85R0A04ZP, UA FA REGISTRATION # FA3RE7RNWP, REGISTERED TO ^PRIVACY DATA OMITTED^ (PIC), REMOTE PILOT CERTIFICATE ^PRIVACY DATA OMITTED^, LOST CONTROLLED FLIGHT IN THE AREA OF ^PRIVACY DATA OMITTED^ AND HIT A BLACK NISSAN PICKUP TRUCK BEARING ^PRIVACY DATA OMITTED^ TRAVELING ALONG TAMIAMI TRAIL IN NORTH PORT CAUSING PROPERTY DAMAGE. THE UAS WAS FLOWN ON A RECREATIONAL FLIGHT OVER A CONSTRUCTION SITE AT ^PRIVACY DATA OMITTED^, USING AN AUTOMATIC FREQUENCY SE

In [144]:
# TODO 4-2 - После вывода информации об инциденте сохраните всю информацию о дроне в файл .json при помощи метода save_data
# ВАШ КОД:
for res in result:
    res.save_data()

# РЕЗУЛЬТАТ:
# см. приложенные файлы в папке samples